In [1]:
from queue import PriorityQueue
from collections import namedtuple
from functools import reduce
from random import random
from math import ceil
import numpy


In [2]:
def covered(state):
    return reduce(
        numpy.logical_or, [SETS[i] for i in state.taken], numpy.array([False for _ in range(PROBLEM_SIZE)])
    )
def goal_check(state):
    return numpy.all(covered(state) )

In [12]:
State = namedtuple('State', ['taken', 'not_taken'])
PROBLEM_SIZE = 15
NUM_SETS = 200
SETS = tuple(numpy.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [17]:
rarityTiles = NUM_SETS - sum(SETS)
estimateperValue = rarityTiles/sum(rarityTiles)
def h(state):
    estimate = sum(estimateperValue * (numpy.logical_not(covered(state))))
    return ceil(estimate)

def h3(state):
    already_covered = covered(state)
    if numpy.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(numpy.logical_and(s, numpy.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

#objective is result with the lowest number of tiles
def f(state):
    return len(state.taken) + h(state)
def f3(state):
    return len(state.taken) + h3(state)

In [18]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state),state))

counter = 0
_ , current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for tile in current_state.not_taken:
        new_state = State(
            current_state.taken ^ {tile}, 
            current_state.not_taken ^ {tile}
            )
        frontier.put((f(new_state), new_state))
    _ , current_state = frontier.get()
    
print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")
print(current_state.taken)

KeyboardInterrupt: 